In [1]:
import requests
import bs4
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# 1. Data Preprocessing

In [2]:
pop = requests.get('https://www.worldometers.info/world-population/population-by-country/')

In [3]:
pop

<Response [200]>

In [4]:
pop.text

'\n\n<!DOCTYPE html><!--[if IE 8]> <html lang="en" class="ie8"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie9"> <![endif]--><!--[if !IE]><!--> <html lang="en"> <!--<![endif]--> <head> <meta charset="utf-8"> <meta http-equiv="X-UA-Compatible" content="IE=edge"> <meta name="viewport" content="width=device-width, initial-scale=1"> <title>Population by Country (2019) - Worldometers</title><meta name="description" content="List of countries and dependencies in the world ranked by population, from the most populated. Growth rate, median age, fertility rate, area, density, population density, urbanization, urban population, share of world population."><!-- Favicon --><link rel="shortcut icon" href="/favicon/favicon.ico" type="image/x-icon"><link rel="apple-touch-icon" sizes="57x57" href="/favicon/apple-icon-57x57.png"><link rel="apple-touch-icon" sizes="60x60" href="/favicon/apple-icon-60x60.png"><link rel="apple-touch-icon" sizes="72x72" href="/favicon/apple-icon-72x72.png"><link rel

In [5]:
soup = BeautifulSoup(pop.text, 'lxml')

In [6]:
soup

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie9"> <![endif]--><!--[if !IE]><!--><html lang="en"> <!--<![endif]--> <head> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1" name="viewport"/> <title>Population by Country (2019) - Worldometers</title><meta content="List of countries and dependencies in the world ranked by population, from the most populated. Growth rate, median age, fertility rate, area, density, population density, urbanization, urban population, share of world population." name="description"/><!-- Favicon --><link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/><link href="/favicon/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/><link 

In [7]:
table = soup.find('table', {'id': 'example2'})

In [8]:
rows = table.find_all('tr')
rows

[<tr> <th>#</th> <th>Country (or dependency)</th> <th>Population<br/> (2019)</th> <th>Yearly<br/> Change</th> <th>Net<br/> Change</th> <th>Density<br/> (P/Km²)</th> <th>Land Area<br/> (Km²)</th> <th>Migrants<br/> (net)</th> <th>Fert.<br/> Rate</th> <th>Med.<br/> Age</th> <th>Urban<br/> Pop %</th> <th>World<br/> Share</th> </tr>,
 <tr> <td>1</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/china-population/">China</a></td> <td style="font-weight: bold;">1,420,062,022</td> <td>0.35 %</td> <td>5,016,094</td> <td>151</td> <td>9,388,211</td> <td>-324,919</td> <td>1.6</td> <td>39</td> <td>60 %</td> <td>18.41 %</td> </tr>,
 <tr> <td>2</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/india-population/">India</a></td> <td style="font-weight: bold;">1,368,737,513</td> <td>1.08 %</td> <td>14,685,659</td> <td>460</td> <td>2,973,190</td> <td>-490,000</td> <td>2.3</td> <td>28</td> <td>34 %</td> <td>17.74 %</t

In [9]:
rows = rows[1:]

In [10]:
rank=[]
country_name=[]
population=[]
yearly_change=[]
net_change=[]
density=[]
land_area=[]
migrants=[]
fert_rate=[]
med_age=[]
urbon_pop=[]
world_share=[]
for row in rows:
    rank.append(row.find_all('td')[0].text)
    country_name.append(row.find_all('td')[1].text)
    population.append(row.find_all('td')[2].text)
    yearly_change.append(row.find_all('td')[3].text)
    net_change.append(row.find_all('td')[4].text)
    density.append(row.find_all('td')[5].text)
    land_area.append(row.find_all('td')[6].text)
    migrants.append(row.find_all('td')[7].text)
    fert_rate.append(row.find_all('td')[8].text)
    med_age.append(row.find_all('td')[9].text)
    urbon_pop.append(row.find_all('td')[10].text)
    world_share.append(row.find_all('td')[11].text)

In [11]:
df = pd.DataFrame()

In [12]:
df['rank']=rank
df['country_name']=country_name
df['population']=population
df['yearly_change']=yearly_change
df['net_change']=net_change
df['density']=density
df['land_area']=land_area
df['migrants']=migrants
df['fert_rate']=fert_rate
df['med_age']=med_age
df['urbon_pop']=urbon_pop
df['world_share']=world_share

In [13]:
df

,rank,country_name,population,yearly_change,net_change,density,land_area,migrants,fert_rate,med_age,urbon_pop,world_share
0,1,China,"1,420,062,022",0.35 %,"5,016,094",151,"9,388,211","-324,919",1.6,39,60 %,18.41 %
1,2,India,"1,368,737,513",1.08 %,"14,685,659",460,"2,973,190","-490,000",2.3,28,34 %,17.74 %
2,3,United States,"329,093,110",0.71 %,"2,326,362",36,"9,147,420","900,000",1.9,38,84 %,4.27 %
3,4,Indonesia,"269,536,482",1.03 %,"2,741,502",149,"1,811,570","-165,000",2.3,29,56 %,3.49 %
4,5,Brazil,"212,392,717",0.72 %,"1,524,763",25,"8,358,140","6,000",1.7,33,86 %,2.75 %
5,6,Pakistan,"204,596,442",1.88 %,"3,782,624",265,"770,880","-214,356",3.4,23,40 %,2.65 %
6,7,Nigeria,"200,962,417",2.60 %,"5,087,180",221,"910,770","-60,000",5.4,18,52 %,2.60 %
7,8,Bangladesh,"168,065,920",1.02 %,"1,697,771","1,291","130,170","-470,000",2.1,27,37 %,2.18 %
8,9,Russia,"143,895,551",-0.05 %,"-69,158",9,"16,376,870","160,000",1.8,40,73 %,1.87 %
9,10,Mexico,"132,328,035",1.20 %,"1,568,961",68,"1,943,950","-60,000",2.1,29,79 %,1.72 %


In [14]:
df.to_csv("ds2.csv", index = False)